The goal of this notebook is to demonstrate conversion of visibility and image data converted between spectral, spatial, and temporal reference frames using the next generation CASA infrastructure.

Ideally, we could follow the quantities interface familiar to casacore users by treating our units as a property of the data, wrapping dask arrays with [pint](https://pint.readthedocs.io/en/stable/) quantities, optionally using a custom-defined context (for domain-specific units e.g., Jy/Beam) or relying on astropy's quantity implementation to [handle NEP18-compliant arrays](https://github.com/pydata/xarray/issues/525#issuecomment-733754394). For details, see [relevant issues](https://github.com/pydata/xarray/projects/2#card-35851549) and new [standalone package](https://github.com/TomNicholas/pint-xarray) for ongoing open-source concept demonstration efforts being undertaken within the pydata ecosystem.

That is not sufficiently mature or supported yet, so the alternative is to use existing conversion routines that operate on numpy ndarrays and parallelize across dask chunks using mapping functions or custom delayed calls.

Beware -- this section of the documentation is still under active development and the code cells in this notebook will probably all throw some form of Exception if executed.

# Reference Frames

## topo_to_lsrk

The following psuedocode definition of this representative function should eventually produce parallelized calls to a `topo_to_lsrk` (or analagous conversion) routine with array size corresponding to the chunk size of `(time,baseline,pol)`, for all channels.

```python
def topo_to_lsrk(input_array)
    # call SkyCoord and other methods
    output_array = function(input_array)
    return output_array(axis=-1, keepdims=True)
```

Then we would call it like

```python
test = xarray.apply_ufunc(topo_to_lsrk, xds.DATA.chunk({'chan':-1}), input_core_dims=[['chan']], dask='parallelized', output_dtypes=[xds.DATA.dtype])
output = test.compute()
```

This approach requires finding or creating a function that operates on numpy arrays instead of the astropy ndarray subclass or custom objects.

### Helper functions

These are probably utilities that we will want elsewhere in CNGI, but for now they can only live inside this one function definition excluded from the public API.

A spectral reference requires time information in addition to spatial coordinates. Some string of the form `'2019-04-24T02:32:10'` is required, and to be in the TOPO frame (e.g., ALMA), this should be the start of the observation.

In [ ]:
def _reference_time(input_xds):

  import datetime

  # the data variables expected by this function come from the global_xds

  try:
    reference_time = input_xds['ASDM_startValidTime'][0]
    reference_time = input_xds.OBS_TIME_RANGE.values[0]
  except:
    reference_time = datetime.datetime.now()

  return reference_time

We'll need to specify the absolute position of observer and target to make use of the frame transformation methods. Luckily this information is usually carried somewhere in our data model and if it isn't, there are ways to access reasonable defaults.

In [ ]:
def _reference_location(input_xds, reference_time):

  from astropy.coordinates import EarthLocation
  from astropy.time import Time

  # the data variables expected by this function come from the global_xds

  try:
    # astropy.time.Time accepts string input of the form '2019-04-24T02:32:10'
    observer_location = EarthLocation.of_site(input_xds['OBS_TELESCOPE_NAME']).get_itrs(obstime=Time(reference_time))
  except:
    # use a crude reference position selection algorithm
    # could also try accessing the'ASDM_POSITION' data variable
    observer_location = EarthLocation(input_xds['ANT_POSITION'].mean()).get_itrs(obstime=Time(reference_time))

  return observer_location

Not only is the reference location of the "observer" required, the source properties must also be defined in the same frame of a form like `SkyCoord('04h21m59.43s +19d32m06.4', frame='icrs', radial_velocity=23.9 * u.km / u.s, distance=144.321 * u.pc)`.

TODO: assign coordinate variables to d1, d2, d3 in vis.zarr

In [ ]:
def _target_location(input_xds):

  from astropy.coordinates import Angle, SkyCoord
  from astropy import units as u
  
  try:
    # assuming these direction axes are sensibly and consistently defined in radians
    # either way, this will be far easier for images
    radec_string = Angle(input_xds.SRC_DIRECTION * u.rad, input_xds.SRC_DIRECTION * u.rad)
  except:
    radec = '04h21m59.43s +19d32m06.4'

  try:
    # telescope dependent... is this is kept anywhere in the ASDM/MS/image formats?
    target_frame ='icrs'
  except:
    # epoch lookup or assume J2000
    target_frame = 'FK5'

  try:
    # another assumption lacking coordinate associated with the d2 dimension
    recession = global_xds.SRC_PROPER_MOTION * u.Hz
  except:
    # set a weird assumed default
    recession = 23.9 * u.km / u.s

  # examples...need to get these from somehwere in the dataset, likely global_xds
  try:
    distance = something
  except:
    distance = 144.321 * u.pc

  target_location =  SkyCoord(radec, frame=target_frame, radial_velocity=recession, distance=distance)

  return target_location


### Using blocked algorithms

Function to apply across the chunks of our visibility dataset in order to avail ourselves of xarray/dask parallelism

In [ ]:
!pip install --quiet astropy --upgrade
import astropy
astropy.__version__

'4.0.1.post1'

Unless this is >= v4.1 we won't be able to use [SpectralCoord class](https://docs.astropy.org/en/latest/coordinates/spectralcoord.html#reference-frame-transformations), but here is how it could look

In [ ]:
def _change_frame(input_array, place, source):
  from astropy import units as u
  from astropy.coordinates import EarthLocation, SkyCoord, SpectralCoord
  from astropy.time import Time
  import xarray
  import numpy as np

  # the input_array will act on the relevant xr.DataArray
  SpectralCoord(input_array, 
                unit=u(img_xds.attrs['rest_frequency'].split('')[:-1]),
                observer=place, 
                target=source,
                #doppler_reference=img_xds.attrs['spectral__reference'], 
                #doppler_convention=img_xds.attrs['velocity__type'])
                )

  output_array = input_array.with_observer_stationary_relative_to('lsrk')

  return output_array(axis=-1, keepdims=True)

### Main function

Tying all the helper functions together would then look something like this

In [ ]:
# the main function to tie all that together would then look something like
def topo_to_lsrk(xds, ddi, reference_time, observer_location, target_location):
  import xarray
  from cngi.dio import read_vis

  vis_xds = cngi.dio.read_vis(xds, ddi=ddi)
  global_xds = cngi.dio.read_vis(xds, ddi='global')

  time = _reference_time(global_xds)
  place = _reference_location(global_xds, reference_time)
  source = _target_location(global_xds, ddi)

  output_xds = xarray.apply_ufunc(change_frame, place, source, vis_xds.DATA.chunk({'chan':-1}), input_core_dims=[['chan']], dask='parallelized', output_dtypes=[vis_xds.DATA.dtype])

  # update some properties of global_xds after conversion?

  # ouptut_xds.compute()
  return output_xds

The previously described approach, following [official demos](http://learn.astropy.org/rst-tutorials/Coordinates-Transform.html) and other example notebooks from astropy docs using SkyCoord objects, currently yields

```python
TypeError: Position information stored on coordinate frame is insufficient to do a full-space position transformation (representation class: <class 'astropy.coordinates.representation.UnitSphericalRepresentation'>)
```
Work ongoing to figure out why and make it run as expected.

### Implementation details

astropy adopts a standard definition of the LSR following [Schönrich et al. 2010](https://ui.adsabs.harvard.edu/abs/2010MNRAS.403.1829S/abstract). 

CASA supports a number of [spectral reference frames](https://casa.nrao.edu/casadocs/latest/memo-series/reference-material/spectral-frames) and the definition of LSR is encoded [in casacore](https://casacore.github.io/casacore/classcasacore_1_1MRadialVelocity.html#a42d02cf6a267b07f66e4c0b2e49d05d3) and seems to follow the references [published by Frank Ghigo at GBO](http://www.gb.nrao.edu/~fghigo/gbtdoc/doppler.html).



## Spatial Coordinates

Currently it is assumed that all image coordinates have native units of radians, which should allow for consistent selection and conversion between different spatial quantities at the ngCASA and/or application layer.

In [ ]:
from astropy import units as u
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import SpectralCoord
from astropy.coordinates import AltAz

These astropy modules allow us to manipulate quantities for unit-aware computation, including transformation between the reference frames in which data are represented.

In [ ]:
VLA_lat = 34.1*u.degree
VLA_lon = -107.6*u.degree
VLA_alt = 2114.89*u.m
print((VLA_lat, VLA_lon, VLA_alt))

observing_location = EarthLocation(lat=VLA_lat, 
                                   lon=VLA_lon, 
                                   height=VLA_alt,
                                   ellipsoid='WGS84')
print(observing_location, type(observing_location))

(<Quantity 34.1 deg>, <Quantity -107.6 deg>, <Quantity 2114.89 m>)
(-1599173.52082635, -5041233.67723585, 3556822.79344969) m <class 'astropy.coordinates.earth.EarthLocation'>


Space and time are notoriously difficult to untangle, but since we can define locations, coordinates on the celestial sphere, and times, we can transform between these quantities.

In [ ]:
reference_time = Time(['2019-10-4T00:00:00'], 
                        format='isot',
                        scale='utc', 
                        location=observing_location)
print(reference_time, type(reference_time))

phase_center = SkyCoord(ra='19h59m28.5s', dec='+40d44m01.5s', frame='fk5')
print(phase_center)

['2019-10-04T00:00:00.000'] <class 'astropy.time.core.Time'>
<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (299.86875, 40.73375)>


Converting between units is then as straightforward as:

In [ ]:
alt_az_frame = AltAz(location=observing_location, obstime=reference_time)
print(alt_az_frame)
new_frame = phase_center.transform_to(alt_az_frame)
print()
print("Compare",
      (new_frame.az.radian,
       new_frame.alt.radian),
      "and",
      (new_frame.az.degree, 
       new_frame.alt.degree))

<AltAz Frame (obstime=['2019-10-04T00:00:00.000'], location=(-1599173.52082635, -5041233.67723585, 3556822.79344969) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron)>

Compare (array([1.1522305]), array([1.07222892])) and (array([66.01794453]), array([61.43419156]))


We should ensure we are making use of the [higher-precision mode](https://docs.astropy.org/en/latest/coordinates/velocities.html#precision-of-radial-velocity-correction) when we implement radial velocity (and possibly all) corrections.

It should also be possible to rely on `'axisunits'` or similar keys in the image Dataset attributes to set up a spectral frame for eager computation.

In [ ]:
sc = SpectralCoord(img_xds.DATA.chan, 
                   unit=astropy.units(img_xds.attrs['rest_frequency'].split('')[:-1]),
                   observer=observatory,
                   target= source, 
                   doppler_reference=img_xds.attrs['spectral__reference'],  
                   doppler_convention=img_xds.attrs['velocity__type'])

## Further reading

CASA:
* https://casa.nrao.edu/casadocs/latest/calibration-and-visibility-data/uv-manipulation/manipulating-visibilities-with-mstransform
* https://casacore.github.io/python-casacore/casacore_measures.html#casacore.measures.measures.radialvelocity
* https://casa.nrao.edu/docs/TaskRef/mstransform-task.html
* https://casa.nrao.edu/docs/TaskRef/cvel-task.html

astropy:
* [source code for `icrs_to_lsr`](https://docs.astropy.org/en/stable/_modules/astropy/coordinates/builtin_frames/lsr.html)
* GitHub links relevant to support for SpectralCoord
* https://github.com/astropy/specutils/issues/422
* https://github.com/astropy/specutils/pull/524

kapteyn:
* https://www.astro.rug.nl/software/kapteyn/wcs.html
* https://www.astro.rug.nl/software/kapteyn/spectralbackground.html?highlight=lsrk